In [11]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [13]:
def load_and_preprocess_images(folder_path):
    images = []
    labels = []
    for label_folder in os.listdir(folder_path):
        label_folder_path = os.path.join(folder_path, label_folder)
        if os.path.isdir(label_folder_path):
            for img_file in os.listdir(label_folder_path):
                img_path = os.path.join(label_folder_path, img_file)
                if img_path.lower().endswith('.pgm'):
                    img = Image.open(img_path).convert('L')  # Convert to grayscale
                    img = img.resize((112, 92))  # Resize to your desired input shape
                    img_array = np.array(img)
                    images.append(img_array)
                    labels.append(label_folder)
    return np.array(images), np.array(labels)

# Load data
folder_path = './dataset'
images, labels = load_and_preprocess_images(folder_path)

# Normalize images and add channel dimension
images = images / 255.0
images = np.expand_dims(images, axis=-1)  # Add channel dimension

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)


In [14]:
# Split dataset into train and temp (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(images, encoded_labels, test_size=0.6, random_state=42, stratify=encoded_labels)

# Split temp into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [18]:
def build_cnn_model(input_shape: tuple, num_classes: int) -> Model:
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)  # Change output layer to num_classes
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Determine number of classes
num_classes = len(np.unique(encoded_labels))

# Build and compile the CNN model
cnn_model = build_cnn_model(input_shape, num_classes)
cnn_model.compile(optimizer=Adam(learning_rate=0.0003), loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
def create_data_generator(X, y, batch_size=64, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rescale=1./255,
            zoom_range=0.2,
            rotation_range=5,
            width_shift_range=0.1,
            height_shift_range=0.1,
            brightness_range=[0.8, 1.2],
            horizontal_flip=True,
            fill_mode='nearest'
        )
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow(X, y, batch_size=batch_size, shuffle=True)

# Create dataset generators
train_ds = create_data_generator(X_train, y_train, augment=True)
val_ds = create_data_generator(X_val, y_val, augment=False)


In [19]:
from tensorflow.keras.utils import to_categorical

# Encode labels as one-hot vectors
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_val_encoded = to_categorical(y_val, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)


In [20]:
def create_data_generator(X, y, batch_size=64, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rescale=1./255,
            zoom_range=0.2,
            rotation_range=5,
            width_shift_range=0.1,
            height_shift_range=0.1,
            brightness_range=[0.8, 1.2],
            horizontal_flip=True,
            fill_mode='nearest'
        )
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow(X, y, batch_size=batch_size, shuffle=True)

# Create dataset generators
train_ds = create_data_generator(X_train, y_train_encoded, augment=True)
val_ds = create_data_generator(X_val, y_val_encoded, augment=False)


In [21]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True, 
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=5, 
    verbose=1, 
    min_lr=1e-6
)

# Train the CNN model
history = cnn_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping, reduce_lr],
    verbose=2
)


Epoch 1/50


c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 - 5s - 2s/step - accuracy: 0.0063 - loss: 3.6890 - val_accuracy: 0.0167 - val_loss: 3.6889 - learning_rate: 3.0000e-04
Epoch 2/50
3/3 - 1s - 449ms/step - accuracy: 0.0188 - loss: 3.6889 - val_accuracy: 0.0167 - val_loss: 3.6889 - learning_rate: 3.0000e-04
Epoch 3/50
3/3 - 2s - 507ms/step - accuracy: 0.0125 - loss: 3.6889 - val_accuracy: 0.0250 - val_loss: 3.6889 - learning_rate: 3.0000e-04
Epoch 4/50
3/3 - 1s - 479ms/step - accuracy: 0.0250 - loss: 3.6889 - val_accuracy: 0.0250 - val_loss: 3.6889 - learning_rate: 3.0000e-04
Epoch 5/50
3/3 - 1s - 429ms/step - accuracy: 0.0250 - loss: 3.6889 - val_accuracy: 0.0000e+00 - val_loss: 3.6889 - learning_rate: 3.0000e-04
Epoch 6/50

Epoch 6: ReduceLROnPlateau reducing learning rate to 6.000000284984708e-05.
3/3 - 1s - 430ms/step - accuracy: 0.0250 - loss: 3.6889 - val_accuracy: 0.0250 - val_loss: 3.6889 - learning_rate: 3.0000e-04
Epoch 7/50
3/3 - 1s - 495ms/step - accuracy: 0.0250 - loss: 3.6889 - val_accuracy: 0.0250 - val_loss: 3.6889 - 

In [22]:
from tensorflow.keras.models import Model

# Define a model for feature extraction (remove final classification layer)
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

# Extract features from the training, validation, and test sets
X_train_features = feature_extractor.predict(X_train)
X_val_features = feature_extractor.predict(X_val)
X_test_features = feature_extractor.predict(X_test)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_features, y_train)

# Make predictions
y_val_pred = knn.predict(X_val_features)
y_test_pred = knn.predict(X_test_features)

# Evaluate the KNN classifier
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

# Detailed classification report
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))


Validation Accuracy: 0.6833333333333333
Test Accuracy: 0.7166666666666667
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       1.00      0.67      0.80         3
           2       1.00      0.33      0.50         3
           3       0.43      1.00      0.60         3
           4       0.27      1.00      0.43         3
           5       0.75      1.00      0.86         3
           6       1.00      1.00      1.00         3
           7       1.00      0.67      0.80         3
           8       0.00      0.00      0.00         3
           9       1.00      0.33      0.50         3
          10       1.00      0.67      0.80         3
          11       1.00      1.00      1.00         3
          12       1.00      0.67      0.80         3
          13       0.43      1.00      0.60         3
          14       0.60      1.00      0.75         3
          15       0.38    

c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_features, y_train)

# Make predictions on the test set
y_test_pred = knn.predict(X_test_features)

# Evaluate the KNN classifier
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')  # Use 'weighted' for multi-class
f1 = f1_score(y_test, y_test_pred, average='weighted')  # Use 'weighted' for multi-class

# Print metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

# Detailed classification report
print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred))


Test Accuracy: 0.7167
Test Precision: 0.7835
Test Recall: 0.7167
Test F1 Score: 0.7011

Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       1.00      1.00      1.00         3
           2       1.00      0.67      0.80         3
           3       0.75      1.00      0.86         3
           4       0.27      1.00      0.43         3
           5       0.60      1.00      0.75         3
           6       1.00      1.00      1.00         3
           7       1.00      0.33      0.50         3
           8       0.33      0.33      0.33         3
           9       0.50      0.33      0.40         3
          10       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         3
          13       0.50      1.00      0.67         3
          14       0.60      1.00      0.75         3
          15       

c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rahla\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo